# Cours de Statistique: Chapitre 4
Intervalles de confiance et tests

In [ ]:
# Uncomment the following and run if not installed.
# %pip install -q ipywidgets

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
from scipy.special import ndtri

from ipywidgets import interactive, fixed, widgets

## Intervalles de confiance

### Code

In [ ]:
def distribution_bernoulli(x, theta):
    if x == 0:
        return 1 - theta
    if x == 1:
        return theta
    return 0

def distribution_gaussienne(x: float, moyenne=0., variance=1.):
    return np.exp(-(x - moyenne)**2 / (2 * variance)) / (np.sqrt(2 * np.pi * variance))

def distribution_poisson(k, theta=1.):
    assert theta > 0
    if k < 0:
        return 0
    k = int(k)
    return np.exp(-theta) * theta**k / np.math.factorial(k)

def distribution_uniforme(x: float, theta=1.):
    if np.abs(theta) < 1e-14:
        return 0. if np.abs(x) > 0 else 1.
    return (1 if 0 <= x <= theta else 0) / (theta)

def distribution_exponentielle(x, theta=1.):
    assert theta > 0
    if x < 0:
        return 0
    return theta * np.exp(-theta * x)

In [ ]:
def plot_gaussienne(moyenne, variance=1., show_moyenne=True, show_variance=True, style="b-"):
    X = np.linspace(moyenne - 4. * np.sqrt(variance), moyenne + 4. * np.sqrt(variance), 1000)
    distribution = distribution_gaussienne(X, moyenne, variance)
    plt.plot(X, distribution, style)
    if show_moyenne:
        plt.annotate(text=r"$\mu$",  ha = "center", va="bottom", xy=(moyenne, distribution_gaussienne(moyenne, moyenne, variance)), xytext=(moyenne, 0.), arrowprops={'arrowstyle': '-', 'color': 'green', 'ls': '-'}, color = 'green')
    if show_variance:
        plt.annotate(text="", xy=(moyenne - variance, distribution_gaussienne(moyenne - variance, moyenne, variance)), xytext=(moyenne + variance, distribution_gaussienne(moyenne + variance, moyenne, variance)), arrowprops={'arrowstyle': '<->', 'color': 'red', 'ls': '--'})
        plt.text(moyenne, distribution_gaussienne(moyenne+variance, moyenne, variance), r"$2\sigma$", ha="center", va="center", color='red', bbox={'boxstyle': 'square', 'fc':'w','ec': 'w'})
    plt.ylim(bottom=0)
    
    return X, distribution

In [ ]:
TRUE_MU = 0.
TRUE_SIGMA = 1.
observations = np.random.normal(TRUE_MU, TRUE_SIGMA, size=1000)

In [ ]:
def exemple_intervalle_confiance_gaussienne(
    observations,
    n=10,
    erreur=0.05,
    show_moyenne=True,
    show_moyenne_empirique=True,
    type: str='centre',
):
    n = min(n, len(observations))
    moyenne_empirique = observations[:n].mean()
    # eps = norm.ppf((2-alpha)/2)/np.sqrt(n)
    X, Y = plot_gaussienne(TRUE_MU, TRUE_SIGMA, show_moyenne, show_variance=False)
    if type.casefold() == 'centre':
        eps = TRUE_SIGMA * ndtri(1-erreur/2)/np.sqrt(n)
        plt.annotate(text=r'-ε', ha="center", va="bottom", xy=(moyenne_empirique-eps, distribution_gaussienne(moyenne_empirique-eps, TRUE_MU, TRUE_SIGMA)), xytext=(moyenne_empirique-eps, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')
        plt.annotate(text=r'+ε', ha="center", va="bottom", xy=(moyenne_empirique+eps, distribution_gaussienne(moyenne_empirique+eps, TRUE_MU, TRUE_SIGMA)), xytext=(moyenne_empirique+eps, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')
        # X = np.linspace(max(-2, moyenne_empirique - eps), min(moyenne_empirique + eps, 2), 1000)
        X = X.clip(moyenne_empirique - eps, moyenne_empirique + eps)
    elif type.casefold() == 'droite':
        eps = TRUE_SIGMA * ndtri(1-erreur)/np.sqrt(n)
        plt.annotate(text=r'-ε', ha="center", va="bottom", xy=(moyenne_empirique-eps, distribution_gaussienne(moyenne_empirique-eps, TRUE_MU, TRUE_SIGMA)), xytext=(moyenne_empirique-eps, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')
        X = np.maximum(X, moyenne_empirique - eps)
    elif type.casefold() == 'gauche':
        eps = TRUE_SIGMA * ndtri(1-erreur)/np.sqrt(n)
        plt.annotate(text=r'+ε', ha="center", va="bottom", xy=(moyenne_empirique+eps, distribution_gaussienne(moyenne_empirique+eps, TRUE_MU, TRUE_SIGMA)), xytext=(moyenne_empirique+eps, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')
        X = np.minimum(X, moyenne_empirique + eps)
    plt.fill_between(X, Y, 0, color='red', alpha=.1)
    if show_moyenne_empirique:
        moyenne_empirique = np.mean(observations[:n])
        plt.annotate(text=r"$\hat{\mu}}$",  ha = "center", va="bottom", xy=(moyenne_empirique, distribution_gaussienne(moyenne_empirique, TRUE_MU, TRUE_SIGMA)), xytext=(moyenne_empirique, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')




### Résultats

In [ ]:
interactive(exemple_intervalle_confiance_gaussienne,
            observations = fixed(observations),
            n=widgets.IntSlider(min=1, max=100, step=1, value=10),
            erreur=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.05),
            type=widgets.Dropdown(options=['centre', 'gauche', 'droite'], description='Type:'),
)

## Tests Statistiques

In [ ]:
MU0 = 0.
SIGMA0 = 1.
MU1 = 2
SIGMA1 = 1.
observations = np.random.normal(MU0, SIGMA0, size=1000)

In [ ]:
def exemple_test_gaussienne(
    observations,
    n=10,
    niveau=0.05,
    show_moyenne_empirique=True,
    show_obs=True,
    show_seuil=True,
    risque: str='première espèce',
    show_vraissemblance: int=None,
):
    n = min(n, len(observations))
    moyenne_empirique = observations[:n].mean()
    # variance_empirique = observations[:n].var()
    # eps = norm.ppf((2-alpha)/2)/np.sqrt(n)
    X0, Y0 = plot_gaussienne(MU0, SIGMA0, False, show_variance=False)
    X1, Y1 = plot_gaussienne(MU1, SIGMA1, False, show_variance=False, style='g-')
    max_Y_value = max(Y0.max(), Y1.max())
    seuil = SIGMA0 * ndtri(1 - niveau) / np.sqrt(n)
    if show_seuil:
        plt.annotate(text=r'$s + \mu_0$', ha="center", va="top", xy=((seuil + MU0), max_Y_value), xytext=((seuil + MU0), 0. - 0.01), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '-'}, color = 'red')
        if risque.casefold() == 'première espèce':
            # plt.annotate(text=r's', ha="center", va="top", xy=(np.sqrt(n) * seuil, ), xytext=(seuil, 0. - 0.01), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '-'}, color = 'red')
            plt.fill_between(X0, Y0, 0, X0 - MU0 >= np.sqrt(n) * seuil, color='red', alpha=.1)
        elif risque.casefold() == 'seconde espèce':
            plt.fill_between(X1, Y1, 0, X1 - MU1 <= np.sqrt(n) * (seuil + MU0 - MU1), color='red', alpha=.1)
        elif risque.casefold() == 'puissance':
            plt.fill_between(X1, Y1, 0, X1 - MU1 >= np.sqrt(n) * (seuil + MU0 - MU1), color='red', alpha=.1)
    if show_moyenne_empirique:
        moyenne_empirique = np.mean(observations[:n])
        plt.annotate(text=r"$\hat{\mu}}$",  ha = "center", va="bottom", xy=(moyenne_empirique, distribution_gaussienne(moyenne_empirique, MU0, SIGMA0)), xytext=(moyenne_empirique, 0.), arrowprops={'arrowstyle': '-', 'color': 'm', 'ls': '--'}, color = 'm')
    if show_obs:
        plt.plot(observations[:n], np.zeros_like(observations[:n]), "m.")
        if show_vraissemblance == 0:
            L = 1
            L_x = distribution_gaussienne(observations[:min(n, 20)], MU0, SIGMA0)
            for point, vraissemblance in zip(observations[:min(n, 20)], L_x):
                plt.annotate(text="",  ha = "center", va="bottom", xy=(point, vraissemblance), xytext=(point, 0.), arrowprops={'arrowstyle': '-', 'color': 'm', 'ls': ':'})
            L = L_x.prod()
            plt.title(fr"$L(X_1, ..., X_n, \mu_0) = ${L:.3e}")
        if show_vraissemblance == 1:
            L = 1
            L_x = distribution_gaussienne(observations[:min(n, 20)], MU1, SIGMA1)
            for point, vraissemblance in zip(observations[:min(n, 20)], L_x):
                plt.annotate(text="",  ha = "center", va="bottom", xy=(point, vraissemblance), xytext=(point, 0.), arrowprops={'arrowstyle': '-', 'color': 'm', 'ls': ':'})
            L = L_x.prod()
            plt.title(fr"$L(X_1, ..., X_n, \mu_1) = ${L:.3e}")
    plt.legend([r'H_0', r'H_1'])



Resultats

In [ ]:
interactive(exemple_test_gaussienne,
            observations = fixed(observations),
            n=widgets.IntSlider(min=1, max=100, step=1, value=10),
            niveau=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.05),
            risque=widgets.Dropdown(options=['première espèce', 'seconde espèce', 'puissance'], description='Risque:'),
            show_vraissemblance=widgets.Dropdown(options=[None, 0, 1], description='Vraissemblance:'),
)

In [ ]:
norm.cdf(norm.ppf(1-0.05) - np.sqrt(3) * MU1)